# Forward Kinematics


calculation of the position and orientation of its end-effector frame from its joint coord

![Forward Kinematics Diagram](./assets/Fig4_1.png)

In [41]:
import numpy as np
from scipy.linalg import expm

## Method one ( Denavit-Hartenberg)

In [15]:
# let (x, y, phi) be the cartesian position (x,y) and orientation phi
# of a 3-link planar manipulator with link lengths L1, L2, and L3

# normal way to do this is to use the forward kinematics equations
L1 = 10
L2 = 10
L3 =  5
theta1 =  np.pi/16
theta2 = np.pi*3/8
theta3 =  2*np.pi/16

x = L1*np.cos(theta1)  + L2 *np.cos(theta1+ theta2) + L3*np.cos(theta1 + theta2 + theta3)
y = L1*np.sin(theta1) + L2 * np.sin(theta1 + theta2) + L3* np.sin(theta1 + theta2 + theta3)
phi = theta1 + theta2 + theta3

Let $T_{xy}$ represent the homogeneous transformation matrix from frame $x$ to frame $y$.

Then the complete transformation from frame 0 to frame 4 is given by:

$$
T_{04} = T_{01} \cdot T_{12} \cdot T_{23} \cdot T_{34}
$$


In [17]:
T_01 = np.array([
            [np.cos(theta1), -np.sin(theta1), 0, 0],
            [np.sin(theta1),  np.cos(theta1), 0, 0],
            [0,               0,              1, 0],
            [0,               0,              0, 1]
        ])


T_12 = np.array([
            [np.cos(theta2), -np.sin(theta2), 0, L1],
            [np.sin(theta2),  np.cos(theta2), 0, 0],
            [0,               0,              1, 0],
            [0,               0,              0, 1]
        ])

T_23 = np.array([
            [np.cos(theta3), -np.sin(theta3), 0, L2],
            [np.sin(theta3),  np.cos(theta3), 0, 0],
            [0,               0,              1, 0],
            [0,               0,              0, 1]
        ])

T_34 = np.array([
            [1, 0, 0, L3],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])



T_04 = T_01 @ T_12 @ T_23 @ T_34



In [22]:
print(f"x = {x:.2f}, y = {y:.2f}, phi = {phi:.2f} rad")

print("T_04 =")
print(T_04)

# break down the T_04 into translation and rotation matrices
translation_matrix = T_04[:2, 3]
print(f"\nTranslation Matrix: {translation_matrix}")
print(f"  ↳ x translation = {translation_matrix[0]:.2f}")
print(f"  ↳ y translation = {translation_matrix[1]:.2f}")

rotation_matrix = T_04[:2, :2]
rotation_angle = np.arctan2(rotation_matrix[1, 0], rotation_matrix[0, 0])
print(f"\nRotation Matrix:\n{rotation_matrix}")
print(f"  ↳ rotation angle (rad) = {rotation_angle:.2f}")
print(f"  ↳ rotation angle (deg) = {np.degrees(rotation_angle):.2f}°")

# Validation between T_04 and the forward kinematics equations
print("\nValidation:")
print(f"Translation x == FK x? {bool(np.isclose(translation_matrix[0], x))}")
print(f"Translation y == FK y? {bool(np.isclose(translation_matrix[1], y))}")
print(f"Rotation angle == FK phi? {bool(np.isclose(rotation_angle, phi))}")



x = 10.78, y = 16.66, phi = 1.77 rad
T_04 =
[[-0.19509032 -0.98078528  0.         10.78330441]
 [ 0.98078528 -0.19509032  0.         16.66268243]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]

Translation Matrix: [10.78330441 16.66268243]
  ↳ x translation = 10.78
  ↳ y translation = 16.66

Rotation Matrix:
[[-0.19509032 -0.98078528]
 [ 0.98078528 -0.19509032]]
  ↳ rotation angle (rad) = 1.77
  ↳ rotation angle (deg) = 101.25°

Validation:
Translation x == FK x? True
Translation y == FK y? True
Rotation angle == FK phi? True


## Method two (Product of Exponentials)

let M be the position and orientation of end frame {4} when all joint angles are set to zero ( called either "home" position and "zero" position fo the robot) Then

In [23]:
M = np.array([
    [1, 0, 0, L1+ L2+L3],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])


When arm is strecthed out straight to the right at its zero configuration, 

In [45]:
w3 = np.array([ 0 , 0, 1]) # rotation axis of the end effector in the base frame
# w3[0] = 0 # rotation around x-axis
# w3[1] = 0 # rotation around y-axis
# w3[2] = 1 # rotation around z-axis

q3 = np.array([L1+L2, 0, 0]) # angular velocity of the end effector in the base frame
# position from the base frame to the joint
# q3[0] = L1+L2 # x-axis
# q3[1] = 0 # y-axis
# q3[2] = 0 # z-axis

v3 = -1 * np.cross(w3, q3)


In [46]:
def make_twist_matrix(w,v):
    """Create a twist matrix from a 3D vector."""
    return np.hstack((w, v)).reshape(6,1)

def make_screw_axis_matrix(w,v):
    """Create a screw axis matrix from a 3D vector."""
    return np.array([
        [0, -w[2], w[1], v[0]],
        [w[2], 0, -w[0], v[1]],
        [-w[1], w[0], 0, v[2]],
        [0, 0, 0, 0]
    ])

In [47]:
S3_twist = make_twist_matrix(w3, v3)
S3_twist_matrix = make_screw_axis_matrix(w3, v3)

print("\nS3_twist:")
print(S3_twist)
print("\nS3_twist_matrix:")
print(S3_twist_matrix)



S3_twist:
[[  0]
 [  0]
 [  1]
 [  0]
 [-20]
 [  0]]

S3_twist_matrix:
[[  0  -1   0   0]
 [  1   0   0 -20]
 [  0   0   0   0]
 [  0   0   0   0]]


In [62]:
# Testing T04 calculated from method two and method one

# Parameters:
L1 = 10
L2 = 10
L3 =  5
theta1 =  np.pi/16
theta2 = np.pi*3/8
theta3 = np.pi/16


# Method 1: Forward Kinematics

T_01 = np.array([
            [np.cos(theta1), -np.sin(theta1), 0, 0],
            [np.sin(theta1),  np.cos(theta1), 0, 0],
            [0,               0,              1, 0],
            [0,               0,              0, 1]
        ])


T_12 = np.array([
            [np.cos(theta2), -np.sin(theta2), 0, L1],
            [np.sin(theta2),  np.cos(theta2), 0, 0],
            [0,               0,              1, 0],
            [0,               0,              0, 1]
        ])

T_23 = np.array([
            [np.cos(theta3), -np.sin(theta3), 0, L2],
            [np.sin(theta3),  np.cos(theta3), 0, 0],
            [0,               0,              1, 0],
            [0,               0,              0, 1]
        ])

T_34 = np.array([
            [1, 0, 0, L3],
            [0, 1, 0, 0],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])



T_04_1 = T_01 @ T_12 @ T_23 @ T_34


# Method 2: Using the screw axis and the exponential map
# Define the screw axis for each joint

w3 = np.array([ 0 , 0, 1])
q3 = np.array([L1+L2, 0, 0])
v3 = -1 * np.cross(w3, q3)

w2 = np.array([ 0 , 0, 1])
q2 = np.array([L1, 0, 0])
v2 = -1 * np.cross(w2, q2)

w1 = np.array([ 0 , 0, 1])
q1 = np.array([0, 0, 0])
v1 = -1 * np.cross(w1, q1)

S3_screw = make_screw_axis_matrix(w3, v3)
S2_screw = make_screw_axis_matrix(w2, v2)
S1_screw = make_screw_axis_matrix(w1, v1)
# T_04_2 =  expm(S3_screw * theta3) @ M 
# or 

def make_exp_matrix(S, theta):
    """Create the exponential matrix from a screw axis and angle."""
    size = S.shape[0]
    return np.eye(size) + np.sin(theta) * S + (1 - np.cos(theta)) * S @ S

T_04_2 =  make_exp_matrix(S1_screw, theta1) @ make_exp_matrix(S2_screw, theta2) @ make_exp_matrix(S3_screw, theta3) @ M

if np.allclose(T_04_1, T_04_2, atol=1e-6):
    print("✅ T_04_1 and T_04_2 match!")
else:
    print("❌ T_04_1 and T_04_2 differ.")
print("T_04_1:\n", T_04_1)
print("T_04_2:\n", T_04_2)
print("Difference:\n", T_04_1 - T_04_2)

✅ T_04_1 and T_04_2 match!
T_04_1:
 [[ 8.32667268e-17 -1.00000000e+00  0.00000000e+00  1.17587560e+01]
 [ 1.00000000e+00  8.32667268e-17  0.00000000e+00  1.67587560e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_04_2:
 [[ 8.32667268e-17 -1.00000000e+00  0.00000000e+00  1.17587560e+01]
 [ 1.00000000e+00  8.32667268e-17  0.00000000e+00  1.67587560e+01]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Difference:
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


Joint n is displaced to some joint value $theta_{n}$. The end-effector frame M then undergoes a displacement of the form

$$
T = e^{[S_n] \theta_n} M
$$


To calculate this, we need:
1) end-effector configuration M @ home position
2) the screw axes $S_{1},....,S_{n}$ 
3) the joint variables $theta_{1},...,theta_{n}$


## example 


![Forward Kinematics Diagram](./assets/Fig4_2.png)


In [63]:
R = np.array([
    [0, 0, 1],
    [0, 1, 0],
    [-1, 0, 0]
])

t = np.array([L1, 0, -L2])

# Construct the homogeneous transformation matrix M
M = np.block([
    [R, t.reshape(3, 1)],
    [np.zeros((1, 3)), np.array([[1]])]
])

In [64]:
w1 = np.array([0, 0, 1])
q1 = np.array([0, 0, 0])


w2 = np.array([0, -1, 0])
q2 = np.array([L1, 0, 0])


w3 = np.array([1, 0, 0])
q3 = np.array([L1, 0, -L2])

v1 = -np.cross(w1, q1)
v2 = -np.cross(w2, q2)
v3 = -np.cross(w3, q3)

S1_screw = make_screw_axis_matrix(w1, v1)
S2_screw = make_screw_axis_matrix(w2, v2)
S3_screw = make_screw_axis_matrix(w3, v3)



print("\nS1_screw:")
print(S1_screw)
print("\nS2_screw:")
print(S2_screw)
print("\nS3_screw:")
print(S3_screw)


S1_screw:
[[ 0 -1  0  0]
 [ 1  0  0  0]
 [ 0  0  0  0]
 [ 0  0  0  0]]

S2_screw:
[[  0   0  -1   0]
 [  0   0   0   0]
 [  1   0   0 -10]
 [  0   0   0   0]]

S3_screw:
[[  0   0   0   0]
 [  0   0  -1 -10]
 [  0   1   0   0]
 [  0   0   0   0]]


## Second Formulation : Screw Axes in the End-Effector Frame


$$
e^{M^{-1} P M} = M^{-1} e^P M
$$


$$
M e^{M^{-1} P M} =  e^P M
$$


$$
T = M e^{[B_n] \theta_n}
$$

$$
B_n = M^{-1} [S_n] M
$$



i.e

$B_i = [A d_{M-1}] S_i , i = 1, ... , n$


![Forward Kinematics Diagram](./assets/Fig4_4.png)

In [ ]:
# parameters
L= 10
# calculate B
def make_B_matrix(M, S):
    """Create the B matrix from the transformation matrix M and screw axis S."""    
    R = M[:3, :3]
    t = M[:3, 3]

    adj = np.block([
        [R.T, np.zeros((3, 3))],
        [(-R.T @ t) @ R.T, R.T]
    ])

    return adj @ S


w1 = np.array([0, 0, 1])
v1 = np.array([-3*L, 0, 0])

w2 = np.array([0, 1, 0])
v2 = np.array([0, 0, 0])

w3 = np.array([-1, 0, 0])
v3 = np.array([0, 0, -3*L])

w4 = np.array([-1, 0, 0])
v4 = np.array([0, 0, -2*L])

w5 = np.array([-1, 0, 0])
v5 = np.array([0, 0, -L])

w6 = np.array([0, 1, 0])
v6 = np.array([0, 0, 0])


S1 = make_screw_axis_matrix(w1, v1)
S2 = make_screw_axis_matrix(w2, v2)
S3 = make_screw_axis_matrix(w3, v3)
S4 = make_screw_axis_matrix(w4, v4)
S5 = make_screw_axis_matrix(w5, v5)
S6 = make_screw_axis_matrix(w6, v6)


# Calculate the B matrix for each screw axis
B1 = make_B_matrix(M, S1)
B2 = make_B_matrix(M, S2)
B3 = make_B_matrix(M, S3)
B4 = make_B_matrix(M, S4)
B5 = make_B_matrix(M, S5)
B6 = make_B_matrix(M, S6)
